In [6]:
import numpy as np
import pandas as pd
import seaborn as sns
import pytrends
from requests import get

df = pd.read_excel('../movies.xlsx', index_col=0)
df.head()
titles = df.index.tolist()


In [7]:
import json

with open('omdb_data.json', 'r') as f:
     data = json.load(f)
df = pd.read_json('omdb_data.json')
df = df.set_index("Title")
df.head()

,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,Language,...,Metascore,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response
Title,,,,,,,,,,,,,,,,,,,,,
The Age of Adaline,2015,PG-13,24 Apr 2015,112 min,"Drama, Fantasy, Romance",Lee Toland Krieger,"J. Mills Goodloe, Salvador Paskowitz","Blake Lively, Michiel Huisman, Harrison Ford","A young woman, born at the turn of the 20th ce...","English, Portuguese, Italian",...,51,7.2,"186,370",tt1655441,movie,08 Sep 2015,"$42,629,776",N/A,N/A,True
Black Christmas,1974,R,20 Dec 1974,98 min,"Horror, Mystery, Thriller",Bob Clark,Roy Moore,"Olivia Hussey, Keir Dullea, Margot Kidder","During their Christmas break, a group of soror...",English,...,65,7.1,"43,766",tt0071222,movie,05 Dec 2006,N/A,N/A,N/A,True
London,2005,R,03 Jul 2021,92 min,"Drama, Romance",Hunter Richards,Hunter Richards,"Jessica Biel, Chris Evans, Jason Statham",London is a drug laden adventure that centers ...,English,...,24,6.4,"23,890",tt0449061,movie,23 May 2006,"$20,361",N/A,N/A,True
Twisted,2004,R,27 Feb 2004,97 min,"Crime, Drama, Mystery",Philip Kaufman,Sarah Thorp,"Ashley Judd, Samuel L. Jackson, Andy Garcia","Jessica, whose father killed her mother and co...","English, Italian, Spanish",...,26,5.3,"22,435",tt0315297,movie,31 Aug 2004,"$25,198,598","Summit Entertainment, Paramount Pictures",N/A,True
Friends with Kids,2011,R,16 Mar 2012,107 min,"Comedy, Drama, Romance",Jennifer Westfeldt,Jennifer Westfeldt,"Jennifer Westfeldt, Adam Scott, Maya Rudolph",Two best friends decide to have a child togeth...,English,...,55,6.1,"42,670",tt1720616,movie,17 Jul 2012,"$7,251,073",Roadside Attractions,N/A,True


In [71]:
from time import sleep
import random
import logging
from pytrends.exceptions import ResponseError
from pytrends.request import TrendReq
from datetime import datetime
from dateutil.relativedelta import relativedelta
from collections import defaultdict

def get_date_one_year(date_string):
    date_obj = datetime.strptime(date_string, '%d %b %Y')
    new_date_obj = date_obj + relativedelta(years=1)
    new_date_string = new_date_obj.strftime('%Y-%m-%d')

    return "{} {}".format(date_obj.strftime('%Y-%m-%d'), new_date_obj.strftime('%Y-%m-%d'))

timeframe = [get_date_one_year(released_date) for released_date in df["Released"]]

requests_args = {
    'headers': {
        'Cookie': '__utma=10102256.1490800305.1679441551.1679844599.1679844599.2; __utmb=10102256.1.10.1679844599; __utmc=10102256; __utmd=1; __utmt=1; __utmz=10102256.1679844599.1.1.utmcsr=trends.google.com|utmccn=(referral)|utmcmd=referral|utmcct=/; SIDCC=AFvIBn8l-M2TM4WYuiIwgDX_jWecoplbo1fngwm68Ot3wJWqsmjrEuK-QFCNqDx84KOy9MjyOU-4; __Secure-1PSIDCC=AFvIBn8GVxzd1Xva_gW-1-gir4mRsds0dU2dRsQjftSFY5tEjO05NY3_KCgvjZdUbFPLBmP8jFo; __Secure-3PSIDCC=AFvIBn8sidF5kq3DqDKsREglAB0A3iCkc7SA7dpfGpsr_H39n299KWALkdpk1pSzgzMelA2mhv4; _ga_VWZPXDNJJB=GS1.1.1679848310.5.0.1679848310.0.0.0; _gat_gtag_UA_4401283=1; 1P_JAR=2023-3-26-16; NID=511=KpGyZzJSh9Dog7UZ3jafxppUxsTV74MK_aRXXz6f54jZ46s63DvQ7f9tD-Jrn5FxrfxdLMk_1jcJtOj2OfZMUKSkMow1-lw4tXclwxnIlZD6TZbWf3JyBfx61F2pZTFd0_OBMslqKZBVkyNGlwI34UmkSaHuUasmXXRfQLBuypawi4MYvA0JqnyjGgsHDA4LFnvSHnh1ofbKdblWkvfaR4fd5fmmnQhQDyO2DM2z0OPovNc_ygD1E8EgnpZaHaaf1zjb5faDePAD9NWIWNiucXyekOPoxnsd6zp0d0JClKG1hfsQjbUUwjLE7ae0CMVBaiF2WgTVWnOCS6aKJaMD2hpKdQJ4Iyb8IX_pq97XkLd-g3N2r1SNLE7kcie8-YfmzccpR35q4SUsUPhYbkeCQ_MByZcovX5cmEMcX5augKmrljAH6ULxVLCcEeGH9iblIg4UJQfSzw; _ga=GA1.3.1490800305.1679441551; _gid=GA1.3.353626381.1679844591; AEC=AUEFqZd8mLhQBGS0fhetYNWeF2paE0jSO3V3yygv8lmQ2Bv081GFl0p85as; APISID=_7EJ-a5syz9zJgEU/AAT5Peub6PlkFkO5E; HSID=APaE7HbeXzljG9lfq; SAPISID=vWm--1fOhBOhKj1Y/AHX6EI69vaZFY39c4; SID=UggFQiXde0ut5rzblEwpNezi6Pl1se0M1QgafLtQw_SYfEHd1C-AAXa8S8RHuvp9W4y7Vw.; SSID=AIe3EdCRUnqT0ETZI; __Secure-1PAPISID=vWm--1fOhBOhKj1Y/AHX6EI69vaZFY39c4; __Secure-1PSID=UggFQiXde0ut5rzblEwpNezi6Pl1se0M1QgafLtQw_SYfEHdEXbXvug4To1QZZxuiOwwqw.; __Secure-3PAPISID=vWm--1fOhBOhKj1Y/AHX6EI69vaZFY39c4; __Secure-3PSID=UggFQiXde0ut5rzblEwpNezi6Pl1se0M1QgafLtQw_SYfEHdP7A18IAyN6mLGaE9sABhAg.; SEARCH_SAMESITE=CgQI7pcB; OTZ=6952293_52_52_123900_48_436380; OGPC=19031496-1:19031779-1:; CONSENT=PENDING+812; SOCS=CAISHAgBEhJnd3NfMjAyMjA4MTEtMF9SQzMaAmVuIAEaBgiA64WYBg; ANID=AHWqTUntC6syHP0EaAPEEzqXA9xCScNDLDS9xmd-i-cLlzSHT6x6n2LnkvG5STTi',
        'Accept': 'application/json, text/plain, */*, text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Encoding': 'gzip, deflate, br',
        'Host': 'trends.google.com',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.3 Safari/605.1.15',
        'Accept-Language': 'en-GB,en;q=0.9',
        'Referer': 'https://trends.google.com/trends/explore?date=all&q=The%20Age%20of%20Adaline&hl=en-GB',
        'Connection': 'keep-alive'
    }
}

pytrend = TrendReq(requests_args=requests_args)
df_trends = dict()
def get_google_trend(title, timeframe):
    for t, tf in zip(title, timeframe):
        print(t, tf)
        pytrend.build_payload(kw_list=[t], timeframe=tf)
        try:
            df_trends[t] = pytrend.related_queries()
            sleep(10 + random.randint(1, 5)) # Time out to ensure google does not flag

        except ResponseError as e:
            print("Error:", e)
            df_trends[t] = None
            # Invalid request, usually caused by released date being earlier than 2004
            sleep(3) # Time out to ensure google does not flag
            

get_google_trend(titles, timeframe)

The Age of Adaline 2015-04-24 2016-04-24
Black Christmas 1974-12-20 1975-12-20
Error: The request failed: Google returned a response with code 500
London 2021-07-03 2022-07-03
Twisted 2004-02-27 2005-02-27
Friends with Kids 2012-03-16 2013-03-16
Ken Park 2003-04-03 2004-04-03
Error: The request failed: Google returned a response with code 500
Girls Trip 2017-07-21 2018-07-21
The Art of the Steal 2014-06-20 2015-06-20
Macbeth 2015-12-11 2016-12-11
Fatherhood 2021-06-18 2022-06-18
London Has Fallen 2016-03-04 2017-03-04
Anthropoid 2016-09-09 2017-09-09
Me, Myself & Irene 2000-06-23 2001-06-23
Error: The request failed: Google returned a response with code 500
The Last Face 2017-07-28 2018-07-28
The Fly 1986-08-15 1987-08-15
Error: The request failed: Google returned a response with code 500
A Quiet Place 2018-04-06 2019-04-06
God's Not Dead 2 2016-04-01 2017-04-01
Marvel One-Shot: Agent Carter 2013-09-24 2014-09-24
Hamlet 2 2008-08-27 2009-08-27
Where the Truth Lies 2005-10-07 2006-10-07

I Kill Giants 2018-03-23 2019-03-23
Harley Davidson and the Marlboro Man 1991-08-23 1992-08-23
Error: The request failed: Google returned a response with code 500
Junebug 2005-09-16 2006-09-16
The Phantom 1996-06-07 1997-06-07
Error: The request failed: Google returned a response with code 500
A Hidden Life 2020-01-17 2021-01-17
National Treasure: Book of Secrets 2007-12-21 2008-12-21
I Am Mother 2019-06-07 2020-06-07
Awake 2007-11-30 2008-11-30
Book of Dragons 2011-11-15 2012-11-15
What the Health 2017-10-04 2018-10-04


In [76]:
df_trends['The Age of Adaline']['The Age of Adaline']['top'].head(10)

,query,value
0,the age of adaline online,100
1,age of adaline online,97
2,movie the age of adaline,88
3,adaline movie,88
4,movies,56
5,the age of adaline 2015,52
6,the age of adaline watch,44
7,the age of adaline watch online,38
8,watch age of adaline online,37
9,the age of adaline trailer,27


In [183]:
indices = df_trends.keys()
dfs = []
for k in indices:
    if df_trends[k] is not None:
        key = list(df_trends[k].keys())[0]
        df = df_trends[k][key]['top']
        dfs.append(df[df['value'] > 50])
    else:
        dfs.append(df_trends[k])

df_trends_df = pd.concat(dfs, axis=0, keys=indices)
df_trends_df.head(20)

query  value
The Age of Adaline 0  the age of adaline online    100
                   1      age of adaline online     97
                   2   movie the age of adaline     88
                   3              adaline movie     88
                   4                     movies     56
                   5    the age of adaline 2015     52
London             0             london weather    100
                   1                time london     85
                   2                         uk     83
                   3               hotel london     51
Twisted            0             twisted sister    100
                   1              twisted metal     97
                   2               twisted pair     74
                   3             twisted lyrics     72
Friends with Kids  0    movie friends with kids    100
                   1   friends with kids online     91
                   2  friends with kids trailer     85
                   3     friends with kids 2012     83
                   4                     movies     67
                   5    watch friends with kids     65

In [327]:
df_trends_df.index = df_trends_df.index.set_names(['Title', 'No'])
df12 = df_trends_df.copy()
df12 = df12.merge(df_trends_df, how='outer', left_index=True, right_index=True)
df12

query_x  \
Title              No                                                   
The Age of Adaline 0                        the age of adaline online   
                   1                            age of adaline online   
                   2                         movie the age of adaline   
                   3                                    adaline movie   
                   4                                           movies   
...                                                               ...   
Book of Dragons    10                                 game of thrones   
                   11  dungeons and dragons the book of vile darkness   
What the Health    0                               what is the health   
                   1                                   what is health   
                   2                                          what is   

                       value_x  \
Title              No            
The Age of Adaline 0       100   
                   1        97   
                   2        88   
                   3        88   
                   4        56   
...                        ...   
Book of Dragons    10       59   
                   11       57   
What the Health    0       100   
                   1        97   
                   2        96   

                                                              query_y  value_y  
Title              No                                                           
The Age of Adaline 0                        the age of adaline online      100  
                   1                            age of adaline online       97  
                   2                         movie the age of adaline       88  
                   3                                    adaline movie       88  
                   4                                           movies       56  
...                                                               ...      ...  
Book of Dragons    10                                 game of thrones       59  
                   11  dungeons and dragons the book of vile darkness       57  
What the Health    0                               what is the health      100  
                   1                                   what is health       97  
                   2                                          what is       96  

[444 rows x 4 columns]

In [194]:
import json

df_trends_df.index.get_level_values(0).unique().to_list()
df_trends_df.index = df_trends_df.index.set_names(['Title', 'No'])
df_trends_df.to_json("./trends_data_1.json", orient='table')
df_trends_df = pd.read_json("./trends_data_1.json", orient='table')
df_trends_df

query  value
Title              No                                                       
The Age of Adaline 0                        the age of adaline online    100
                   1                            age of adaline online     97
                   2                         movie the age of adaline     88
                   3                                    adaline movie     88
                   4                                           movies     56
...                                                               ...    ...
Book of Dragons    10                                 game of thrones     59
                   11  dungeons and dragons the book of vile darkness     57
What the Health    0                               what is the health    100
                   1                                   what is health     97
                   2                                          what is     96

[444 rows x 2 columns]

In [207]:
requests_args = {
    'headers': {
        'Cookie': '__utma=10102256.1490800305.1679441551.1679844599.1679844599.2; __utmb=10102256.1.10.1679844599; __utmc=10102256; __utmd=1; __utmt=1; __utmz=10102256.1679844599.1.1.utmcsr=trends.google.com|utmccn=(referral)|utmcmd=referral|utmcct=/; SIDCC=AFvIBn8l-M2TM4WYuiIwgDX_jWecoplbo1fngwm68Ot3wJWqsmjrEuK-QFCNqDx84KOy9MjyOU-4; __Secure-1PSIDCC=AFvIBn8GVxzd1Xva_gW-1-gir4mRsds0dU2dRsQjftSFY5tEjO05NY3_KCgvjZdUbFPLBmP8jFo; __Secure-3PSIDCC=AFvIBn8sidF5kq3DqDKsREglAB0A3iCkc7SA7dpfGpsr_H39n299KWALkdpk1pSzgzMelA2mhv4; _ga_VWZPXDNJJB=GS1.1.1679848310.5.0.1679848310.0.0.0; _gat_gtag_UA_4401283=1; 1P_JAR=2023-3-26-16; NID=511=KpGyZzJSh9Dog7UZ3jafxppUxsTV74MK_aRXXz6f54jZ46s63DvQ7f9tD-Jrn5FxrfxdLMk_1jcJtOj2OfZMUKSkMow1-lw4tXclwxnIlZD6TZbWf3JyBfx61F2pZTFd0_OBMslqKZBVkyNGlwI34UmkSaHuUasmXXRfQLBuypawi4MYvA0JqnyjGgsHDA4LFnvSHnh1ofbKdblWkvfaR4fd5fmmnQhQDyO2DM2z0OPovNc_ygD1E8EgnpZaHaaf1zjb5faDePAD9NWIWNiucXyekOPoxnsd6zp0d0JClKG1hfsQjbUUwjLE7ae0CMVBaiF2WgTVWnOCS6aKJaMD2hpKdQJ4Iyb8IX_pq97XkLd-g3N2r1SNLE7kcie8-YfmzccpR35q4SUsUPhYbkeCQ_MByZcovX5cmEMcX5augKmrljAH6ULxVLCcEeGH9iblIg4UJQfSzw; _ga=GA1.3.1490800305.1679441551; _gid=GA1.3.353626381.1679844591; AEC=AUEFqZd8mLhQBGS0fhetYNWeF2paE0jSO3V3yygv8lmQ2Bv081GFl0p85as; APISID=_7EJ-a5syz9zJgEU/AAT5Peub6PlkFkO5E; HSID=APaE7HbeXzljG9lfq; SAPISID=vWm--1fOhBOhKj1Y/AHX6EI69vaZFY39c4; SID=UggFQiXde0ut5rzblEwpNezi6Pl1se0M1QgafLtQw_SYfEHd1C-AAXa8S8RHuvp9W4y7Vw.; SSID=AIe3EdCRUnqT0ETZI; __Secure-1PAPISID=vWm--1fOhBOhKj1Y/AHX6EI69vaZFY39c4; __Secure-1PSID=UggFQiXde0ut5rzblEwpNezi6Pl1se0M1QgafLtQw_SYfEHdEXbXvug4To1QZZxuiOwwqw.; __Secure-3PAPISID=vWm--1fOhBOhKj1Y/AHX6EI69vaZFY39c4; __Secure-3PSID=UggFQiXde0ut5rzblEwpNezi6Pl1se0M1QgafLtQw_SYfEHdP7A18IAyN6mLGaE9sABhAg.; SEARCH_SAMESITE=CgQI7pcB; OTZ=6952293_52_52_123900_48_436380; OGPC=19031496-1:19031779-1:; CONSENT=PENDING+812; SOCS=CAISHAgBEhJnd3NfMjAyMjA4MTEtMF9SQzMaAmVuIAEaBgiA64WYBg; ANID=AHWqTUntC6syHP0EaAPEEzqXA9xCScNDLDS9xmd-i-cLlzSHT6x6n2LnkvG5STTi',
        'Accept': 'application/json, text/plain, */*, text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Encoding': 'gzip, deflate, br',
        'Host': 'trends.google.com',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.3 Safari/605.1.15',
        'Accept-Language': 'en-GB,en;q=0.9',
        'Referer': 'https://trends.google.com/trends/explore?date=all&q=The%20Age%20of%20Adaline&hl=en-GB',
        'Connection': 'keep-alive'
    }
}

pytrend = TrendReq(requests_args=requests_args)
df_trends2 = defaultdict(dict)
def get_google_trend(title, timeframe):
    for t, tf in zip(title, timeframe):
        print(t)
        for query in df_trends_df.loc[t]['query']:
            pytrend.build_payload(kw_list=[query], timeframe=tf)
            try:
                df_trends2[t][query] = pytrend.interest_over_time()
                sleep(10 + random.randint(1, 5)) # Time out to ensure google does not flag

            except ResponseError as e:
                print("Error:", e)
                df_trends2[t][query] = None
                # Invalid request, usually caused by released date being earlier than 2004
                sleep(3)
            print(t, tf, query)


get_google_trend(df_trends_df.index.get_level_values(0).unique().to_list(), timeframe)

The Age of Adaline
The Age of Adaline 2015-04-24 2016-04-24 the age of adaline online
The Age of Adaline 2015-04-24 2016-04-24 age of adaline online
The Age of Adaline 2015-04-24 2016-04-24 movie the age of adaline
The Age of Adaline 2015-04-24 2016-04-24 adaline movie
The Age of Adaline 2015-04-24 2016-04-24 movies
The Age of Adaline 2015-04-24 2016-04-24 the age of adaline 2015
London
Error: The request failed: Google returned a response with code 500
London 1974-12-20 1975-12-20 london weather
Error: The request failed: Google returned a response with code 500
London 1974-12-20 1975-12-20 time london
Error: The request failed: Google returned a response with code 500
London 1974-12-20 1975-12-20 uk
Error: The request failed: Google returned a response with code 500
London 1974-12-20 1975-12-20 hotel london
Twisted
Twisted 2021-07-03 2022-07-03 twisted sister
Twisted 2021-07-03 2022-07-03 twisted metal
Twisted 2021-07-03 2022-07-03 twisted pair
Twisted 2021-07-03 2022-07-03 twisted l

The Art of Self-Defense 2006-12-08 2007-12-08 midsommar
Ninja Assassin
Ninja Assassin 2021-05-14 2022-05-14 torrent ninja assassin
Ninja Assassin 2021-05-14 2022-05-14 ninja assassin online
Ninja Assassin 2021-05-14 2022-05-14 the ninja assassin
Ninja Assassin 2021-05-14 2022-05-14 ninja assassin movie
Ninja Assassin 2021-05-14 2022-05-14 ninja assassin download
Bad Hair Day
Bad Hair Day 2018-07-13 2019-07-13 movie bad hair day
Bad Hair Day 2018-07-13 2019-07-13 the bad hair day
Bad Hair Day 2018-07-13 2019-07-13 bad hair day disney
Bad Hair Day 2018-07-13 2019-07-13 disney
Bad Hair Day 2018-07-13 2019-07-13 full movie bad hair day
Justice League: Doom
Error: The request failed: Google returned a response with code 500
Justice League: Doom 1939-08-25 1940-08-25 the justice league
Error: The request failed: Google returned a response with code 500
Justice League: Doom 1939-08-25 1940-08-25 the justice league doom
Error: The request failed: Google returned a response with code 500
Justic

Saving Mr. Banks 1991-08-23 1992-08-23 saving mr. banks online
Error: The request failed: Google returned a response with code 500
Saving Mr. Banks 1991-08-23 1992-08-23 mr. banks mary poppins
Error: The request failed: Google returned a response with code 500
Saving Mr. Banks 1991-08-23 1992-08-23 mary poppins
Error: The request failed: Google returned a response with code 500
Saving Mr. Banks 1991-08-23 1992-08-23 saving mr. banks 2013
Elegy
Elegy 2016-08-26 2017-08-26 the elegy
Elegy 2016-08-26 2017-08-26 elegy movie
Welcome to Marwen
Error: The request failed: Google returned a response with code 500
Welcome to Marwen 2000-09-01 2001-09-01 welcome to the marwen
Error: The request failed: Google returned a response with code 500
Welcome to Marwen 2000-09-01 2001-09-01 second act
Error: The request failed: Google returned a response with code 500
Welcome to Marwen 2000-09-01 2001-09-01 movie welcome to marwen
Error: The request failed: Google returned a response with code 500
Welcome

The Princess Switch 2006-10-06 2007-10-06 vanessa hudgens princess switch
The Princess Switch 2006-10-06 2007-10-06 the princess switch vanessa hudgens
The Princess Switch 2006-10-06 2007-10-06 vanessa hudgens
The Princess Switch 2006-10-06 2007-10-06 princess movies
The Princess Switch 2006-10-06 2007-10-06 princess switch full movie
The Princess Switch 2006-10-06 2007-10-06 the switch princess full movie
Freaks of Nature
Freaks of Nature 2010-10-01 2011-10-01 movie freaks of nature
Freaks of Nature 2010-10-01 2011-10-01 freaks of nature 2015
Freaks of Nature 2010-10-01 2011-10-01 trailer freaks of nature
Freaks of Nature 2010-10-01 2011-10-01 freaks of nature 2016
The Irishman
The Irishman 2019-05-24 2020-05-24 cast the irishman
The Irishman 2019-05-24 2020-05-24 irishman cast
The Irishman 2019-05-24 2020-05-24 irishman movie
The Irishman 2019-05-24 2020-05-24 movie the irishman
The Irishman 2019-05-24 2020-05-24 hoffa
The Irishman 2019-05-24 2020-05-24 irishman netflix
The Irishman 

Book of Dragons 2019-11-27 2020-11-27 dance with dragons
Book of Dragons 2019-11-27 2020-11-27 book of vile darkness
Book of Dragons 2019-11-27 2020-11-27 a dance of dragons
Book of Dragons 2019-11-27 2020-11-27 dungeons and dragons book of vile darkness
Book of Dragons 2019-11-27 2020-11-27 game of thrones dragons
Book of Dragons 2019-11-27 2020-11-27 a dance with dragons
Book of Dragons 2019-11-27 2020-11-27 game of thrones book
Book of Dragons 2019-11-27 2020-11-27 game of thrones
Book of Dragons 2019-11-27 2020-11-27 dungeons and dragons the book of vile darkness
What the Health
Error: The request failed: Google returned a response with code 500
What the Health 1975-12-25 1976-12-25 what is the health
Error: The request failed: Google returned a response with code 500
What the Health 1975-12-25 1976-12-25 what is health
Error: The request failed: Google returned a response with code 500
What the Health 1975-12-25 1976-12-25 what is


In [209]:
# df_trends_df.index.get_level_values(0).unique().to_list()
df_trends_df

query  value
Title              No                                                       
The Age of Adaline 0                        the age of adaline online    100
                   1                            age of adaline online     97
                   2                         movie the age of adaline     88
                   3                                    adaline movie     88
                   4                                           movies     56
...                                                               ...    ...
Book of Dragons    10                                 game of thrones     59
                   11  dungeons and dragons the book of vile darkness     57
What the Health    0                               what is the health    100
                   1                                   what is health     97
                   2                                          what is     96

[444 rows x 2 columns]

In [221]:
df_trends2['London']['london weather']

In [328]:
indices = df_trends2.keys()
dfs2 = []
for k in indices:
    if df_trends2[k] is not None:
        dfi = []
        keys = df_trends2[k].keys()
        print(k, keys)
        for q in keys:
            if df_trends2[k][q] is not None:
                dfi.append(df_trends2[k][q][q])
        if dfi:
#             dfs2.append(pd.concat(dfi, axis=0, keys=keys))
            dfs2.append(pd.DataFrame(pd.concat(dfi, axis=0, keys=keys)))
        else:
            dfs2.append(None)
    else:
        pass
#          dfs2.append(df_trends2[k])
    

df_trends_df2 = pd.concat(dfs2, axis=0, keys=indices)
df_trends_df2 = df_trends_df2.rename(columns={0: "searches"})
df_trends_df2 = pd.DataFrame(df_trends_df2.loc[..., "searches"])
df_trends_df2.index.names = ['Title', 'Query', 'date']
df_trends_df2

The Age of Adaline dict_keys(['the age of adaline online', 'age of adaline online', 'movie the age of adaline', 'adaline movie', 'movies', 'the age of adaline 2015'])
London dict_keys(['london weather', 'time london', 'uk', 'hotel london'])
Twisted dict_keys(['twisted sister', 'twisted metal', 'twisted pair', 'twisted lyrics'])
Friends with Kids dict_keys(['movie friends with kids', 'friends with kids online', 'friends with kids trailer', 'friends with kids 2012', 'movies', 'watch friends with kids', 'friends with benefits'])
Girls Trip dict_keys(['movie girls trip', 'movie', 'the girls trip', 'girls trip 2017', 'girls trip movies', 'movies'])
The Art of the Steal dict_keys(['the art of the steal movie', 'the art of steal movie', 'the art of the steal 2013', 'art of the steal 2013', 'trailer the art of the steal', 'art of the steal trailer', 'the art of steal trailer', 'the art of the steal 2014', 'the art of the steal film', 'the art of the steal imdb'])
Macbeth dict_keys(['lady macbe

searches
Title              Query                                          date                
The Age of Adaline the age of adaline online                      2015-04-26      71.0
                                                                  2015-05-03      76.0
                                                                  2015-05-10      88.0
                                                                  2015-05-17      93.0
                                                                  2015-05-24     100.0
...                                                                                ...
Book of Dragons    dungeons and dragons the book of vile darkness 2020-10-25       0.0
                                                                  2020-11-01       0.0
                                                                  2020-11-08       0.0
                                                                  2020-11-15      50.0
                                                                  2020-11-22       0.0

[17842 rows x 1 columns]

In [316]:
import json

df_trends_df2.to_json("./trends_data_2.json", orient='table')
trends_data_2 = pd.read_json("./trends_data_2.json", orient='table')

trends_data_2.tail(50)

searches
Title           Query                                          date                
Book of Dragons dungeons and dragons the book of vile darkness 2019-12-15       0.0
                                                               2019-12-22       0.0
                                                               2019-12-29       0.0
                                                               2020-01-05     100.0
                                                               2020-01-12      30.0
                                                               2020-01-19       0.0
                                                               2020-01-26       0.0
                                                               2020-02-02       0.0
                                                               2020-02-09       0.0
                                                               2020-02-16       0.0
                                                               2020-02-23       0.0
                                                               2020-03-01       0.0
                                                               2020-03-08       0.0
                                                               2020-03-15       0.0
                                                               2020-03-22      35.0
                                                               2020-03-29       0.0
                                                               2020-04-05       0.0
                                                               2020-04-12      85.0
                                                               2020-04-19      38.0
                                                               2020-04-26       0.0
                                                               2020-05-03       0.0
                                                               2020-05-10       0.0
                                                               2020-05-17       0.0
                                                               2020-05-24       0.0
                                                               2020-05-31       0.0
                                                               2020-06-07      53.0
                                                               2020-06-14       0.0
                                                               2020-06-21       0.0
                                                               2020-06-28       0.0
                                                               2020-07-05       0.0
                                                               2020-07-12       0.0
                                                               2020-07-19       0.0
                                                               2020-07-26      23.0
                                                               2020-08-02       0.0
                                                               2020-08-09       0.0
                                                               2020-08-16       0.0
                                                               2020-08-23      25.0
                                                               2020-08-30       0.0
                                                               2020-09-06       0.0
                                                               2020-09-13       0.0
                                                               2020-09-20       0.0
                                                               2020-09-27       0.0
                                                               2020-10-04       0.0
                                                               2020-10-11       0.0
                                                               2020-10-18       0.0
                                                               2020-10-25       0.0
                                           

In [315]:
trends_data_2.groupby("Title").sum()

,searches
Title,
A Hidden Life,0.0
A Quiet Place,9481.0
Alpha,34242.0
Anthropoid,5110.0
Ashby,6084.0
...,...
The Turning,5086.0
The Weather Man,2508.0
Twisted,13324.0
